In [ ]:
!pip install ultralytics

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import os
import glob
from datetime import datetime
import time
import cv2
import shutil
from random import random

from ultralytics import YOLO

In [ ]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')


# Import custom module with helper functions
import sys
sys.path.append('/content/drive/My Drive')
import funcs_yolo as f

Mounted at /content/drive


In [ ]:
# Unzip archives with images
!unzip -q "/content/drive/My Drive/raw.zip" -d "/content"

In [ ]:
p = os.path.abspath('.')
input_dir = os.path.join(p, 'raw', 'images')
X_files, binary_labels = f.read_names(input_dir)

In [ ]:
X_train, X_val, y_train, y_val = f.custom_train_val_split(X_files, binary_labels, 0.2, 42 )

Full dataset: 96 images with bacteria and 481 without
Shuffling validation dataset ....
(115,) - (115,)
Shuffling trainig dataset ....
Validation X: (115,); y: (115,)
Training X: (462,); y: (462,)


In [ ]:
def boxesFromYOLO(imagePath,labelPath):
    image = cv2.imread(imagePath)
    (hI, wI) = image.shape[:2]
    lines = [line.rstrip('\n') for line in open(labelPath)]
    boxes = []
    if lines != ['']:
        for line in lines:
            components = line.split(" ")
            category = components[0]
            x  = int(float(components[1])*wI - float(components[3])*wI/2)
            y = int(float(components[2])*hI - float(components[4])*hI/2)
            h = int(float(components[4])*hI)
            w = int(float(components[3])*wI)
            boxes.append((category, (x, y, w, h)))
    return (hI, wI,image,boxes)

def flip_ver(image,boxes,H,W):
    txt_yolo = []
    img= cv2.flip(image,1)
    (H,W) = img.shape[:2]
    for box in boxes:
        class_name = int(box[0])
        (x, y, w, h) = box[1]
        x2 = ((W - x -w)+w/2)/W
        h2 = h/H
        w2 = w/W
        y2 = (y+(h/2))/H
        txt_yolo.append((class_name,round(x2,4),round(y2,4),round(w2,4),round(h2,4)))
    return img,txt_yolo

def flip_hor(image,boxes,H,W):
    txt_yolo = []
    img= cv2.flip(image,0)
    (H,W) = img.shape[:2]
    for box in boxes:
        class_name =int(box[0])
        (x, y, w, h) = box[1]
        x2 = (x+(w/2))/W
        h2 = h/H
        w2 = w/W
        y2 =((H -y -h)+h/2)/H
        txt_yolo.append((class_name,round(x2,4),round(y2,4),round(w2,4),round(h2,4)))
    return img,txt_yolo

def brightness_augment(img,boxes,H,W):
    hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV) #convert to hsv
    hsv = np.array(hsv, dtype=np.float64)
    hsv[:, :, 2] = hsv[:, :, 2] * (1 + np.random.uniform(-0.2, 0.2)) #scale channel V uniformly
    hsv[:, :, 2][hsv[:, :, 2] > 255] = 255 #reset out of range values
    rgb = cv2.cvtColor(np.array(hsv, dtype=np.uint8), cv2.COLOR_HSV2RGB)
    (H,W) = img.shape[:2]
    txt_yolo = []
    for box in boxes:
        class_name =int(box[0])
        (x, y, w, h) = box[1]
        x2 = (x+(w/2))/W
        h2 = h/H
        w2 = w/W
        y2 =(y+(h/2))/H
        txt_yolo.append((class_name,round(x2,4),round(y2,4),round(w2,4),round(h2,4)))
    return rgb,txt_yolo


In [ ]:
train_img_path = os.path.join(p, 'drive', 'MyDrive', 'YOLO', 'data', 'images','train')
os.makedirs(train_img_path)
val_img_path = os.path.join(p, 'drive', 'MyDrive', 'YOLO', 'data', 'images','val')
os.makedirs(val_img_path)
train_lbl_path = os.path.join(p, 'drive', 'MyDrive', 'YOLO', 'data', 'labels','train')
os.makedirs(train_lbl_path)
val_lbl_path = os.path.join(p, 'drive', 'MyDrive', 'YOLO', 'data', 'labels','val')
os.makedirs(val_lbl_path)

In [ ]:
for val_file in X_val:
  img_curr_path = os.path.join(p, 'raw', 'images', val_file)
  img_new_path = os.path.join(val_img_path, val_file)
  shutil.move(img_curr_path, img_new_path)
  lbl_curr_path = os.path.join(p, 'raw', 'labels', val_file[:-3] +'txt')
  lbl_new_path = os.path.join(val_lbl_path, val_file[:-3] +'txt')
  shutil.move(lbl_curr_path, lbl_new_path)

In [ ]:
for train_file in X_train:
  img_curr_path = os.path.join(p, 'raw', 'images', train_file)
  lbl_curr_path = os.path.join(p, 'raw', 'labels', train_file[:-3] +'txt')
  hI, wI,image,boxes = boxesFromYOLO(img_curr_path, lbl_curr_path)

  img_fv,txt_yolo_fv = flip_ver(image,boxes,hI,wI)

  fv_im_path = os.path.join(train_img_path, train_file.split('-')[0]+'-'+train_file.split('-')[1]+'-fv-'+train_file.split('-')[2])
  fv_lbl_path = os.path.join(train_lbl_path, train_file.split('-')[0]+'-'+train_file.split('-')[1]+'-fv-'+train_file.split('-')[2][:-3]+'txt')

  cv2.imwrite(fv_im_path, img_fv)
  with open(fv_lbl_path, 'w') as f:
      if txt_yolo_fv != []:
          for box in txt_yolo_fv:
              f.write(f'{box[0]} {box[1]} {box[2]} {box[3]} {box[4]}\n')
      else:
          f.write('')

  img_fh,txt_yolo_fh = flip_hor(image,boxes,hI,wI)

  fh_im_path = os.path.join(train_img_path, train_file.split('-')[0]+'-'+train_file.split('-')[1]+'-fh-'+train_file.split('-')[2])
  fh_lbl_path = os.path.join(train_lbl_path, train_file.split('-')[0]+'-'+train_file.split('-')[1]+'-fh-'+train_file.split('-')[2][:-3]+'txt')

  cv2.imwrite(fh_im_path, img_fh)
  with open(fh_lbl_path, 'w') as f:
      if txt_yolo_fh != []:
          for box in txt_yolo_fh:
              f.write(f'{box[0]} {box[1]} {box[2]} {box[3]} {box[4]}\n')
      else:
          f.write('')

  img_br,txt_yolo_br = brightness_augment(image,boxes,hI,wI)

  br_im_path = os.path.join(train_img_path, train_file.split('-')[0]+'-'+train_file.split('-')[1]+'-br-'+train_file.split('-')[2])
  br_lbl_path = os.path.join(train_lbl_path, train_file.split('-')[0]+'-'+train_file.split('-')[1]+'-br-'+train_file.split('-')[2][:-3]+'txt')

  cv2.imwrite(br_im_path, img_br)
  with open(br_lbl_path, 'w') as f:
      if txt_yolo_br != []:
          for box in txt_yolo_br:
              f.write(f'{box[0]} {box[1]} {box[2]} {box[3]} {box[4]}\n')
      else:
          f.write('')

  img_new_path = os.path.join(train_img_path, train_file)
  shutil.move(img_curr_path, img_new_path)
  lbl_new_path = os.path.join(train_lbl_path, train_file[:-3] +'txt')
  shutil.move(lbl_curr_path, lbl_new_path)


In [ ]:
model = YOLO('yolov8n.pt')

100%|██████████| 6.23M/6.23M [00:00<00:00, 115MB/s]


In [ ]:
result = model.train(data  = os.path.join(p, 'drive', 'MyDrive', 'YOLO', 'data.yaml'), epochs = 50)

Ultralytics YOLOv8.1.0 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/drive/MyDrive/YOLO/data.yaml, epochs=50, time=None, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=Tr

100%|██████████| 755k/755k [00:00<00:00, 27.0MB/s]


Overriding model.yaml nc=80 with nc=9

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

train: Scanning /content/drive/MyDrive/YOLO/data/labels/train... 1848 images, 24 backgrounds, 0 corrupt: 100%|██████████| 1848/1848 [00:07<00:00, 236.74it/s]


train: New cache created: /content/drive/MyDrive/YOLO/data/labels/train.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


...

      50/50      2.32G     0.8619     0.4466      0.961         18        640: 100%|██████████| 116/116 [01:14<00:00,  1.55it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:01<00:00,  2.35it/s]

                   all        115        347      0.787      0.867      0.874      0.532



50 epochs completed in 1.130 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 6.3MB
Optimizer stripped from runs/detect/train/weights/best.pt, 6.3MB

Validating runs/detect/train/weights/best.pt...
Ultralytics YOLOv8.1.0 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3007403 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:04<00:00,  1.12s/it]


                   all        115        347      0.754       0.88      0.884      0.544
                bubble        115         60      0.698      0.617      0.677      0.287
                colony        115         77      0.909      0.935      0.946      0.612
                  gate        115         19      0.892      0.895      0.922      0.624
                  lock        115         75      0.894          1      0.953      0.668
               sharpie        115         25      0.768        0.8      0.811      0.492
                  star        115          1      0.296          1      0.995      0.497
                  tape        115         18      0.688      0.833      0.849      0.491
                unlock        115         72      0.887      0.958      0.916       0.68
Speed: 0.3ms preprocess, 3.4ms inference, 0.0ms loss, 8.1ms postprocess per image
Results saved to runs/detect/train


In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

!scp -r /content/runs /content/drive/MyDrive/YOLO